<div class="alert alert-block alert-info">
__Name__: pangenome_feature_selection<br/>
__Description__: Try out feature selection methods<br/>
__Author__: Matthew Whiteside matthew dot whiteside at canada dot ca<br/>
__Date__: Oct 23, 2017<br/>
__TODO__:<br/>
</div>

In [1]:
from hpsklearn import HyperoptEstimator, any_classifier, xgboost_classification, random_forest, gradient_boosting, extra_trees
from hyperopt import tpe
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib
matplotlib.rcParams['figure.figsize'] = [10.0,8.0]

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
os.chdir('../pangenome')
import config
import utils
import classify
pg, genome_list, locus_list = utils.read_panseq(config.PANSEQ['pangenome_file'])
amr, amr_list = utils.read_amr(config.PHENOTYPE['amr_file'], genome_list)
annot = utils.read_annot(config.ANNOTATION['blast_file'])

In [67]:
# Split into train & test for ampicillin
d = np.argwhere(amr_list == 'ampicillin').item(0)
validrows = ~np.isnan(amr[:,d])
validrows
X = pg[validrows,:]
y = amr[validrows,d]

test_size = int( 0.2 * len( y ) )
np.random.seed( 2123 )
indices = np.random.permutation(X.shape[0])
X_train = X[ indices[:-test_size] ]
y_train = y[ indices[:-test_size] ]
X_test = X[ indices[-test_size:] ]
y_test = y[ indices[-test_size:] ]

In [47]:
# Define loss function
def loss_fn(y_target, y_prediction):
    return 1.0 - f1_score(y_target, y_prediction)
# HP search for Random Forest
rfc = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000)
rfc.fit( X_train.toarray(), y_train )
print( rfc.score( X_test.toarray(), y_test ) )
print( rfc.best_model() )
predictions = rfc.predict( X_test.toarray() )
print(classification_report(y_test, predictions))


0.867647058824
{'learner': RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=4,
            max_features=0.766719831648635, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=3,
            verbose=False, warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
             precision    recall  f1-score   support

        0.0       0.60      0.75      0.67        12
        1.0       0.94      0.89      0.92        56

avg / total       0.88      0.87      0.87        68



In [7]:
rfcb = rfc._best_learner
rfcb

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features=0.8493329707313447,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1535, n_jobs=1,
            oob_score=False, random_state=0, verbose=False,
            warm_start=False)

In [96]:
# HP search for Gradient Boost
gbc = HyperoptEstimator( classifier=gradient_boosting('gbc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000)
gbc.fit( X_train.toarray(), y_train )
print( gbc.score( X_test.toarray(), y_test ) )
print( gbc.best_model() )
predictions = gbc.predict( X_test.toarray() )
print(classification_report(y_test, predictions))

0.867647058824
{'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.005308968883120171, loss='deviance',
              max_depth=None, max_features=0.2973197785199644,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=562,
              presort='auto', random_state=1, subsample=1.0, verbose=0,
              warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
             precision    recall  f1-score   support

        0.0       0.62      0.67      0.64        12
        1.0       0.93      0.91      0.92        56

avg / total       0.87      0.87      0.87        68



In [97]:
gbcb = gbc._best_learner
gbcb

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.005308968883120171, loss='deviance',
              max_depth=None, max_features=0.2973197785199644,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=562,
              presort='auto', random_state=1, subsample=1.0, verbose=0,
              warm_start=False)

In [32]:
# Try feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

In [42]:
fs = SelectKBest(chi2, k=2000)
X_new_train = fs.fit_transform(X_train,y_train)
X_new_test = fs.transform(X_test)
rfc2 = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000)
rfc2.fit( X_new_train.toarray(), y_train )
print( rfc2.score( X_new_test.toarray(), y_test ) )
print( rfc2.best_model() )
predictions2 = rfc2.predict( X_new_test.toarray() )
print(classification_report(y_test, predictions2))

0.955882352941
{'learner': RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None,
            max_features=0.4452771242357403, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=19, n_jobs=1, oob_score=False, random_state=1,
            verbose=False, warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
             precision    recall  f1-score   support

        0.0       0.78      0.88      0.82         8
        1.0       0.98      0.97      0.97        60

avg / total       0.96      0.96      0.96        68



In [41]:
fs = SelectKBest(f_classif, k=2000)
X_new_train = fs.fit_transform(X_train,y_train)
X_new_test = fs.transform(X_test)
rfc2 = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000)
rfc2.fit( X_new_train.toarray(), y_train )
print( rfc2.score( X_new_test.toarray(), y_test ) )
print( rfc2.best_model() )
predictions2 = rfc2.predict( X_new_test.toarray() )
print(classification_report(y_test, predictions2))

/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
/home/matt/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


0.941176470588
{'learner': RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features='log2',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2784, n_jobs=1,
            oob_score=False, random_state=1, verbose=False,
            warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
             precision    recall  f1-score   support

        0.0       1.00      0.50      0.67         8
        1.0       0.94      1.00      0.97        60

avg / total       0.94      0.94      0.93        68



In [48]:
fs = SelectKBest(mutual_info_classif, k=2000)
X_new_train = fs.fit_transform(X_train,y_train)
X_new_test = fs.transform(X_test)
rfc2 = HyperoptEstimator( classifier=random_forest('rfc'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000)
rfc2.fit( X_new_train.toarray(), y_train )
print( rfc2.score( X_new_test.toarray(), y_test ) )
print( rfc2.best_model() )
predictions2 = rfc2.predict( X_new_test.toarray() )
print(classification_report(y_test, predictions2))

0.852941176471
{'learner': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.35008535557916565,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=11, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=170, n_jobs=1,
            oob_score=False, random_state=3, verbose=False,
            warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
             precision    recall  f1-score   support

        0.0       0.56      0.75      0.64        12
        1.0       0.94      0.88      0.91        56

avg / total       0.88      0.85      0.86        68



In [100]:
# Try stacked implementation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [69]:
X_train_rf, X_train_lr, y_train_rf, y_train_lr = train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.5)

In [70]:
# Supervised transformation based on gradient boosting
rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features=0.8493329707313447,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1535, n_jobs=4,
            oob_score=False, random_state=0, verbose=False,
            warm_start=False)
rf_enc = OneHotEncoder()
rf_lm = LogisticRegression()
rf.fit(X_train_rf, y_train_rf)
rf_enc.fit(rf.apply(X_train_rf))
rf_lm.fit(rf_enc.transform(rf.apply(X_train_lr)), y_train_lr, n_jobs=4)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [76]:
y_pred_rf_lm = rf_lm.predict(rf_enc.transform(rf.apply(X_test)))
y_pred_rf = rf.predict(X_test)

In [78]:
print(classification_report(y_test, y_pred_rf_lm))

             precision    recall  f1-score   support

        0.0       0.62      0.67      0.64        12
        1.0       0.93      0.91      0.92        56

avg / total       0.87      0.87      0.87        68



In [77]:
print(classification_report(y_test, y_pred_rf))

             precision    recall  f1-score   support

        0.0       0.80      0.33      0.47        12
        1.0       0.87      0.98      0.92        56

avg / total       0.86      0.87      0.84        68



In [80]:
confusion_matrix(y_test, y_pred_rf_lm)

array([[ 8,  4],
       [ 5, 51]])

In [104]:
grd = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.005308968883120171, loss='deviance',
              max_depth=None, max_features=0.2973197785199644,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=562,
              presort='auto', random_state=1, subsample=1.0, verbose=0,
              warm_start=False)
grd_enc = OneHotEncoder()
grd_lm = LogisticRegression()
grd.fit(X_train, y_train)
grd_enc.fit(grd.apply(X_train)[:, :, 0])
grd_lm.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [107]:
y_pred_grd_lm = grd_lm.predict(grd_enc.transform(grd.apply(X_test)[:, :, 0]))
y_pred_grd = grd.predict(X_test.toarray())
print(classification_report(y_test, y_pred_grd_lm))
print(classification_report(y_test, y_pred_grd))


             precision    recall  f1-score   support

        0.0       0.69      0.75      0.72        12
        1.0       0.95      0.93      0.94        56

avg / total       0.90      0.90      0.90        68

             precision    recall  f1-score   support

        0.0       0.62      0.67      0.64        12
        1.0       0.93      0.91      0.92        56

avg / total       0.87      0.87      0.87        68



In [110]:
etc = HyperoptEstimator( classifier=extra_trees('et'), preprocessing=[], algo=tpe.suggest, loss_fn=loss_fn, trial_timeout=2000)
etc.fit( X_train.toarray(), y_train )
print( etc.score( X_test.toarray(), y_test ) )
print( etc.best_model() )
predictions = etc.predict( X_test.toarray() )
print(classification_report(y_test, predictions))

0.838235294118
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features=0.4105662978479582,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=2982, n_jobs=1,
           oob_score=False, random_state=2, verbose=False,
           warm_start=False), 'preprocs': (), 'ex_preprocs': ()}
             precision    recall  f1-score   support

        0.0       0.53      0.75      0.62        12
        1.0       0.94      0.86      0.90        56

avg / total       0.87      0.84      0.85        68

